# Homework and bake-off: word-level entailment with neural networks

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Fall 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Data](#Data)
1. [Baseline](#Baseline)
  1. [Representing words: vector_func](#Representing-words:-vector_func)
  1. [Combining words into inputs: vector_combo_func](#Combining-words-into-inputs:-vector_combo_func)
  1. [Classifier model](#Classifier-model)
  1. [Baseline results](#Baseline-results)
1. [Homework questions](#Homework-questions)
  1. [Hypothesis-only baseline [2 points]](#Hypothesis-only-baseline-[2-points])
  1. [Alternatives to concatenation [2 points]](#Alternatives-to-concatenation-[2-points])
  1. [A deeper network [2 points]](#A-deeper-network-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

The general problem is word-level natural language inference. Training examples are pairs of words $(w_{L}, w_{R}), y$ with $y = 1$ if $w_{L}$ entails $w_{R}$, otherwise $0$.

The homework questions below ask you to define baseline models for this and develop your own system for entry in the bake-off, which will take place on a held-out test-set distributed at the start of the bake-off. (Thus, all the data you have available for development is available for training your final system before the bake-off begins.)

## Set-up

See [the first notebook in this unit](nli_01_task_and_data.ipynb) for set-up instructions.

In [2]:
from collections import defaultdict
import json
import numpy as np
import os
import pandas as pd
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
import nli
import utils

ModuleNotFoundError: No module named 'torch_shallow_neural_classifier'

In [ ]:
DATA_HOME = 'data'

NLIDATA_HOME = os.path.join(DATA_HOME, 'nlidata')

wordentail_filename = os.path.join(
    NLIDATA_HOME, 'nli_wordentail_bakeoff_data.json')

GLOVE_HOME = os.path.join(DATA_HOME, 'glove.6B')

## Data

I've processed the data into a train/dev split that is designed to put some pressure on our models to actually learn these semantic relations, as opposed to exploiting regularities in the sample. 

The defining feature of the dataset is that the `train` and `dev` __vocabularies__ are disjoint. That is, if a word `w` appears in a training pair, it does not occur in any text pair. It follows from this that there are also no word-pairs shared between train and dev, as you would expect. This should require your models to learn abstract relationships, as opposed to memorizing incidental properties of individual words in the dataset.

In [ ]:
with open(wordentail_filename) as f:
    wordentail_data = json.load(f)

The keys are the splits plus a list giving the vocabulary for the entire dataset:

In [5]:
wordentail_data.keys()

dict_keys(['dev', 'train', 'vocab'])

In [6]:
wordentail_data['train'][: 5]

[[['abode', 'house'], 1],
 [['abortion', 'anaemia'], 0],
 [['abortion', 'aneurysm'], 0],
 [['abortion', 'blindness'], 0],
 [['abortion', 'deafness'], 0]]

In [7]:
nli.get_vocab_overlap_size(wordentail_data)

0

Because no words are shared between `train` and `dev`, no pairs are either:

In [8]:
nli.get_pair_overlap_size(wordentail_data)

0

Here is the label distribution:

In [9]:
pd.DataFrame(wordentail_data['train'])[1].value_counts()

0    7000
1    1283
Name: 1, dtype: int64

This is a challenging label distribution – there are more than 5 times as more non-entailment cases as entailment cases.

## Baseline

Even in deep learning, __feature representation is vital and requires care!__ For our task, feature representation has two parts: representing the individual words and combining those representations into a single network input.

### Representing words: vector_func

Let's consider two baseline word representations methods:

1. Random vectors (as returned by `utils.randvec`).
1. 50-dimensional GloVe representations.

In [10]:
def randvec(w, n=50, lower=-1.0, upper=1.0):
    """Returns a random vector of length `n`. `w` is ignored."""
    return utils.randvec(n=n, lower=lower, upper=upper)

In [11]:
def load_glove50():
    glove_src = os.path.join(GLOVE_HOME, 'glove.6B.50d.txt')
    # Creates a dict mapping strings (words) to GloVe vectors:
    GLOVE = utils.glove2dict(glove_src)
    return GLOVE

GLOVE = load_glove50()

def glove_vec(w):
    """Return `w`'s GloVe representation if available, else return
    a random vector."""
    return GLOVE.get(w, randvec(w, n=50))

### Combining words into inputs: vector_combo_func

Here we decide how to combine the two word vectors into a single representation. In more detail, where `u` is a vector representation of the left word and `v` is a vector representation of the right word, we need a function `vector_combo_func` such that `vector_combo_func(u, v)` returns a new input vector `z` of dimension `m`. A simple example is concatenation:

In [12]:
def vec_concatenate(u, v):
    """Concatenate np.array instances `u` and `v` into a new np.array"""
    return np.concatenate((u, v))

`vector_combo_func` could instead be vector average, vector difference, etc. (even combinations of those) – there's lots of space for experimentation here; [homework question 2](#Alternatives-to-concatenation-[2-points]) below pushes you to do some exploration.

### Classifier model

For a baseline model, I chose `TorchShallowNeuralClassifier`:

In [13]:
net = TorchShallowNeuralClassifier(early_stopping=True)

### Baseline results

The following puts the above pieces together, using `vector_func=glove_vec`, since `vector_func=randvec` seems so hopelessly misguided for our problem!

In [14]:
baseline_experiment = nli.wordentail_experiment(
    train_data=wordentail_data['train'],
    assess_data=wordentail_data['dev'],
    model=net,
    vector_func=glove_vec,
    vector_combo_func=vec_concatenate)

Stopping after epoch 131. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 1.0115265101194382

              precision    recall  f1-score   support

           0      0.871     0.948     0.908      1732
           1      0.500     0.269     0.350       334

    accuracy                          0.838      2066
   macro avg      0.685     0.609     0.629      2066
weighted avg      0.811     0.838     0.818      2066



## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Hypothesis-only baseline [2 points]

During our discussion of SNLI and MultiNLI, we noted that a number of research teams have shown that hypothesis-only baselines for NLI tasks can be remarkably robust. This question asks you to explore briefly how this baseline affects our task.

For this problem, submit two functions:

1. A `vector_combo_func` function called `hypothesis_only` that simply throws away the premise, using the unmodified hypothesis (second) vector as its representation of the example.

1. A function called `run_hypothesis_only_evaluation` that does the following:
    1. Loops over the two `vector_combo_func` values `vec_concatenate` and `hypothesis_only`, calling `nli.wordentail_experiment` to train on the 'train' portion and assess on the 'dev' portion, with `glove_vec` as the `vector_func`. So that the results are consistent, use an `sklearn.linear_model.LogisticRegression` with default parameters as the model.
    1. Returns a `dict` mapping `function_name` strings to the 'macro-F1' score for that pair, as returned by the call to `nli.wordentail_experiment`. (Tip: you can get the `str` name of, e.g., `hypothesis_only` with `hypothesis_only.__name__`.)
    
The functions `test_hypothesis_only` and `test_run_hypothesis_only_evaluation` will help ensure that your functions have the desired logic.

In [15]:
def hypothesis_only(u, v):
    ##### YOUR CODE HERE
    return v


def run_hypothesis_only_evaluation():
    ##### YOUR CODE HERE
    results = {}
    for vector_combo_function in [vec_concatenate, hypothesis_only]:
        print(vector_combo_function.__name__)
        result = nli.wordentail_experiment(train_data=wordentail_data['train'],
                                  assess_data=wordentail_data['dev'],
                                  model=net,
                                  vector_func=glove_vec,
                                  vector_combo_func=vector_combo_function)
        results[vector_combo_function.__name__] = result['macro-F1']
    return results

In [16]:
def test_hypothesis_only(hypothesis_only):
    v = hypothesis_only(1, 2)
    assert v == 2

In [17]:
test_hypothesis_only(hypothesis_only)

In [18]:
def test_run_hypothesis_only_evaluation(run_hypothesis_only_evaluation):
    results = run_hypothesis_only_evaluation()
    assert all(x in results for x in ('hypothesis_only', 'vec_concatenate')), \
        ("The return value of `run_hypothesis_only_evaluation` does not "
         "have the intended kind of keys.")
    assert isinstance(results['vec_concatenate'], float), \
        ("The values of the `run_hypothesis_only_evaluation` result "
         "should be floats.")

In [19]:
test_run_hypothesis_only_evaluation(run_hypothesis_only_evaluation)

vec_concatenate


Stopping after epoch 122. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 1.2040377408266068

              precision    recall  f1-score   support

           0      0.871     0.960     0.913      1732
           1      0.557     0.263     0.358       334

    accuracy                          0.847      2066
   macro avg      0.714     0.612     0.635      2066
weighted avg      0.820     0.847     0.823      2066

hypothesis_only


Stopping after epoch 127. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 2.1055303812026978

              precision    recall  f1-score   support

           0      0.852     0.974     0.909      1732
           1      0.483     0.126     0.200       334

    accuracy                          0.837      2066
   macro avg      0.668     0.550     0.554      2066
weighted avg      0.793     0.837     0.794      2066



### Alternatives to concatenation [2 points]

We've so far just used vector concatenation to represent the premise and hypothesis words. This question asks you to explore two simple alternative:

1. Write a function `vec_diff` that, for a given pair of vector inputs `u` and `v`, returns the element-wise difference between `u` and `v`.

1. Write a function `vec_max` that, for a given pair of vector inputs `u` and `v`, returns the element-wise max values between `u` and `v`.

You needn't include your uses of `nli.wordentail_experiment` with these functions, but we assume you'll be curious to see how they do!

In [20]:
def vec_diff(u, v):
    ##### YOUR CODE HERE
    return u - v


def vec_max(u, v):
    ##### YOUR CODE HERE
    return np.maximum(u, v)



In [21]:
def test_vec_diff(vec_diff):
    u = np.array([10.2, 8.1])
    v = np.array([1.2, -7.1])
    result = vec_diff(u, v)
    expected = np.array([9.0, 15.2])
    assert np.array_equal(result, expected), \
        "Expected {}; got {}".format(expected, result)

In [22]:
test_vec_diff(vec_diff)

In [23]:
def test_vec_max(vec_max):
    u = np.array([1.2,  8.1])
    v = np.array([10.2, -7.1])
    result = vec_max(u, v)
    expected = np.array([10.2, 8.1])
    assert np.array_equal(result, expected), \
        "Expected {}; got {}".format(expected, result)

In [24]:
test_vec_max(vec_max)

In [25]:
def run_multiple_evaluation():
    results = {}
    for vector_combo_function in [vec_concatenate, hypothesis_only, vec_diff, vec_max]:
        print(vector_combo_function.__name__)
        result = nli.wordentail_experiment(train_data=wordentail_data['train'],
                                  assess_data=wordentail_data['dev'],
                                  model=net,
                                  vector_func=glove_vec,
                                  vector_combo_func=vector_combo_function)
        results[vector_combo_function.__name__] = result['macro-F1']
    return results

In [26]:
run_multiple_evaluation()

vec_concatenate


Stopping after epoch 41. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 2.4612676799297333

              precision    recall  f1-score   support

           0      0.866     0.948     0.905      1732
           1      0.471     0.240     0.317       334

    accuracy                          0.833      2066
   macro avg      0.668     0.594     0.611      2066
weighted avg      0.802     0.833     0.810      2066

hypothesis_only


Stopping after epoch 43. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 2.6879835426807404

              precision    recall  f1-score   support

           0      0.852     0.965     0.905      1732
           1      0.417     0.129     0.197       334

    accuracy                          0.830      2066
   macro avg      0.635     0.547     0.551      2066
weighted avg      0.782     0.830     0.791      2066

vec_diff


Stopping after epoch 20. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 3.0221438705921173

              precision    recall  f1-score   support

           0      0.853     0.945     0.897      1732
           1      0.358     0.159     0.220       334

    accuracy                          0.818      2066
   macro avg      0.606     0.552     0.558      2066
weighted avg      0.773     0.818     0.788      2066

vec_max


Stopping after epoch 34. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 2.7981352508068085

              precision    recall  f1-score   support

           0      0.857     0.958     0.905      1732
           1      0.443     0.174     0.249       334

    accuracy                          0.831      2066
   macro avg      0.650     0.566     0.577      2066
weighted avg      0.790     0.831     0.799      2066



{'vec_concatenate': 0.6113211179363329,
 'hypothesis_only': 0.5509015522863168,
 'vec_diff': 0.5584516569089979,
 'vec_max': 0.5771445997577933}

### A deeper network [2 points]

It is very easy to subclass `TorchShallowNeuralClassifier` if all you want to do is change the network graph: all you have to do is write a new `build_graph`. If your graph has new arguments that the user might want to set, then you should also redefine `__init__` so that these values are accepted and set as attributes.

For this question, please subclass `TorchShallowNeuralClassifier` so that it defines the following graph:

$$\begin{align}
h_{1} &= xW_{1} + b_{1} \\
r_{1} &= \textbf{Bernoulli}(1 - \textbf{dropout_prob}, n) \\
d_{1} &= r_1 * h_{1} \\
h_{2} &= f(d_{1}) \\
h_{3} &= h_{2}W_{2} + b_{2}
\end{align}$$

Here, $r_{1}$ and $d_{1}$ define a dropout layer: $r_{1}$ is a random binary vector of dimension $n$, where the probability of a value being $1$ is given by $1 - \textbf{dropout_prob}$. $r_{1}$ is multiplied element-wise by our first hidden representation, thereby zeroing out some of the values. The result is fed to the user's activation function $f$, and the result of that is fed through another linear layer to produce $h_{3}$. (Inside `TorchShallowNeuralClassifier`, $h_{3}$ is the basis for a softmax classifier; no activation function is applied to it because the softmax scaling is handled internally by the loss function.)

For your implementation, please use `nn.Sequential`, `nn.Linear`, and `nn.Dropout` to define the required layers.

For comparison, using this notation, `TorchShallowNeuralClassifier` defines the following graph:

$$\begin{align}
h_{1} &= xW_{1} + b_{1} \\
h_{2} &= f(h_{1}) \\
h_{3} &= h_{2}W_{2} + b_{2}
\end{align}$$

The following code starts this sub-class for you, so that you can concentrate on `build_graph`. Be sure to make use of `self.dropout_prob`.

For this problem, submit just your completed  `TorchDeepNeuralClassifier`. You needn't evaluate it, though we assume you will be keen to do that!

You can use `test_TorchDeepNeuralClassifier` to ensure that your network has the intended structure.

In [27]:
import torch.nn as nn

class TorchDeepNeuralClassifier(TorchShallowNeuralClassifier):
    def __init__(self, dropout_prob=0.7, **kwargs):
        self.dropout_prob = dropout_prob
        super().__init__(**kwargs)

    def build_graph(self):
        """Complete this method!

        Returns
        -------
        an `nn.Module` instance, which can be a free-standing class you
        write yourself, as in `torch_rnn_classifier`, or the outpiut of
        `nn.Sequential`, as in `torch_shallow_neural_classifier`.

        """
    
        ##### YOUR CODE HERE
        return nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.Dropout(self.dropout_prob),
            self.hidden_activation,
            nn.Linear(self.hidden_dim, self.n_classes_))


In [28]:
def test_TorchDeepNeuralClassifier(TorchDeepNeuralClassifier):
    dropout_prob = 0.55
    assert hasattr(TorchDeepNeuralClassifier(), "dropout_prob"), \
        "TorchDeepNeuralClassifier must have an attribute `dropout_prob`."
    try:
        inst = TorchDeepNeuralClassifier(dropout_prob=dropout_prob)
    except TypeError:
        raise TypeError("TorchDeepNeuralClassifier must allow the user "
                        "to set `dropout_prob` on initialization")
    inst.input_dim = 10
    inst.n_classes_ = 5
    graph = inst.build_graph()
    assert len(graph) == 4, \
        "The graph should have 4 layers; yours has {}".format(len(graph))
    expected = {
        0: 'Linear',
        1: 'Dropout',
        2: 'Tanh',
        3: 'Linear'}
    for i, label in expected.items():
        name = graph[i].__class__.__name__
        assert label in name, \
            ("The {} layer of the graph should be a {} layer; "
            "yours is {}".format(i, label, name))
    assert graph[1].p == dropout_prob, \
        ("The user's value for `dropout_prob` should be the value of "
         "`p` for the Dropout layer.")

In [29]:
test_TorchDeepNeuralClassifier(TorchDeepNeuralClassifier)

In [30]:
assert hasattr(TorchDeepNeuralClassifier(), "dropout_prob")

In [31]:
deep_net = TorchDeepNeuralClassifier(dropout_prob=0.5)
def run_multiple_evaluation():
    results = {}
    for vector_combo_function in [vec_concatenate, hypothesis_only, vec_diff, vec_max]:
        print(vector_combo_function.__name__)
        result = nli.wordentail_experiment(train_data=wordentail_data['train'],
                                  assess_data=wordentail_data['dev'],
                                  model=deep_net,
                                  vector_func=glove_vec,
                                  vector_combo_func=vector_combo_function)
        results[vector_combo_function.__name__] = result['macro-F1']
    return results

In [32]:
run_multiple_evaluation()

vec_concatenate


Stopping after epoch 52. Training loss did not improve more than tol=1e-05. Final error is 2.9124351143836975.

              precision    recall  f1-score   support

           0      0.864     0.957     0.908      1732
           1      0.493     0.216     0.300       334

    accuracy                          0.837      2066
   macro avg      0.678     0.586     0.604      2066
weighted avg      0.804     0.837     0.810      2066

hypothesis_only


Stopping after epoch 60. Training loss did not improve more than tol=1e-05. Final error is 3.160431295633316.

              precision    recall  f1-score   support

           0      0.852     0.978     0.911      1732
           1      0.506     0.117     0.190       334

    accuracy                          0.839      2066
   macro avg      0.679     0.547     0.550      2066
weighted avg      0.796     0.839     0.794      2066

vec_diff


Stopping after epoch 38. Training loss did not improve more than tol=1e-05. Final error is 3.521256744861603.

              precision    recall  f1-score   support

           0      0.848     0.983     0.911      1732
           1      0.500     0.087     0.148       334

    accuracy                          0.838      2066
   macro avg      0.674     0.535     0.529      2066
weighted avg      0.792     0.838     0.787      2066

vec_max


Stopping after epoch 61. Training loss did not improve more than tol=1e-05. Final error is 3.295184165239334.

              precision    recall  f1-score   support

           0      0.851     0.965     0.904      1732
           1      0.408     0.126     0.192       334

    accuracy                          0.829      2066
   macro avg      0.630     0.545     0.548      2066
weighted avg      0.780     0.829     0.789      2066



{'vec_concatenate': 0.6039978094194961,
 'hypothesis_only': 0.5501444749370803,
 'vec_diff': 0.5293271854196224,
 'vec_max': 0.5483425867722787}

### Your original system [3 points]

This is a simple dataset, but its "word-disjoint" nature ensures that it's a challenging one, and there are lots of modeling strategies one might adopt. 

You are free to do whatever you like. We require only that your system differ in some way from those defined in the preceding questions. They don't have to be completely different, though. For example, you might want to stick with the model but represent examples differently, or the reverse.

You are free to use different pretrained word vectors and the like.

Please embed your code in this notebook so that we can rerun it.

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.  We also ask that you report the best score your system got during development, just to help us understand how systems performed overall.

In [33]:
class MYClassifier(TorchShallowNeuralClassifier):
    def __init__(self, dropout_prob=0.7, **kwargs):
        super().__init__(**kwargs)
        self.dropout_prob = dropout_prob
        self.params += ['dropout_prob']

    def build_graph(self):
        """Complete this method!

        Returns
        -------
        an `nn.Module` instance, which can be a free-standing class you
        write yourself, as in `torch_rnn_classifier`, or the outpiut of
        `nn.Sequential`, as in `torch_shallow_neural_classifier`.

        """
    
        ##### YOUR CODE HERE
        return nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.Dropout(self.dropout_prob),
            self.hidden_activation,
            nn.Linear(self.hidden_dim, self.n_classes_))
    
    def build_dataset(self, X, y=None):
        """
        """
        X = np.array(X)
        self.input_dim = X.shape[1]
        
        if y is None:
            X = torch.FloatTensor(X)
            dataset = torch.utils.data.TensorDataset(X)
        else:
            self.classes_ = sorted(set(y))
            self.n_classes_ = len(self.classes_)
            class2index = dict(zip(self.classes_, range(self.n_classes_)))
            y = [class2index[label] for label in y]
            X, y = oversample.fit_resample(X, y)
            X = torch.FloatTensor(X)
            y = torch.tensor(y)
            dataset = torch.utils.data.TensorDataset(X, y)
        return dataset

    

In [34]:
# def load_glove300():
#     glove_src = os.path.join(GLOVE_HOME, 'glove.6B.300d.txt')
#     # Creates a dict mapping strings (words) to GloVe vectors:
#     GLOVE = utils.glove2dict(glove_src)
#     return GLOVE

# GLOVE = load_glove300()

# def glove_vec(w):
#     """Return `w`'s GloVe representation if available, else return
#     a random vector."""
#     return GLOVE.get(w, randvec(w, n=300))

In [35]:
#model = MYClassifier(dropout_prob=0.4, eta=0.001, hidden_dim=300)

In [36]:
# nli.wordentail_experiment(train_data=wordentail_data['train'],
#                                   assess_data=wordentail_data['dev'],
#                                   model=model,
#                                   vector_func=glove_vec,
#                                   vector_combo_func=vec_concatenate)

In [38]:
import torch

In [41]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# IMPORT ANY MODULES BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# The original system is built using a one-hidden layer neural classifier with dropout.
# The input consist of the concatenation of the embedding (GLoVe 300d) of the two words.
# As a baseline I used as embedding GloVe 50d and estimated the performance.
# The final model is trained using GLoVe 300d, additionally, I used the SMOTE technique to handle class imbalance.


class MYClassifier(TorchShallowNeuralClassifier):
    def __init__(self, dropout_prob=0.7, **kwargs):
        super().__init__(**kwargs)
        self.dropout_prob = dropout_prob
        self.params += ['dropout_prob']

    def build_graph(self):
        """Complete this method!

        Returns
        -------
        an `nn.Module` instance, which can be a free-standing class you
        write yourself, as in `torch_rnn_classifier`, or the outpiut of
        `nn.Sequential`, as in `torch_shallow_neural_classifier`.

        """
    
        ##### YOUR CODE HERE
        return nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.Dropout(self.dropout_prob),
            self.hidden_activation,
            nn.Linear(self.hidden_dim, self.n_classes_))
    
    def build_dataset(self, X, y=None):
        """
        """
        X = np.array(X)
        self.input_dim = X.shape[1]
        
        if y is None:
            X = torch.FloatTensor(X)
            dataset = torch.utils.data.TensorDataset(X)
        else:
            self.classes_ = sorted(set(y))
            self.n_classes_ = len(self.classes_)
            class2index = dict(zip(self.classes_, range(self.n_classes_)))
            y = [class2index[label] for label in y]
            X, y = oversample.fit_resample(X, y)
            X = torch.FloatTensor(X)
            y = torch.tensor(y)
            dataset = torch.utils.data.TensorDataset(X, y)
        return dataset

# def load_glove300():
#     glove_src = os.path.join(GLOVE_HOME, 'glove.6B.300d.txt')
#     # Creates a dict mapping strings (words) to GloVe vectors
#     GLOVE = utils.glove2dict(glove_src)
#     return GLOVE

# GLOVE = load_glove300()

# def glove_vec(w):
#     """Return `w`'s GloVe representation if available, else return
#     a random vector."""
#     return GLOVE.get(w, randvec(w, n=300))

# My peak score was: 0.658 
if 'IS_GRADESCOPE_ENV' not in os.environ:
    from imblearn.over_sampling import SMOTE
    oversample = SMOTE()
    def load_glove300():
        glove_src = os.path.join(GLOVE_HOME, 'glove.6B.300d.txt')
    # Creates a dict mapping strings (words) to GloVe vectors
        GLOVE = utils.glove2dict(glove_src)
        return GLOVE

    GLOVE = load_glove300()

    def glove_vec(w):
        """Return `w`'s GloVe representation if available, else return
        a random vector."""
        return GLOVE.get(w, randvec(w, n=300))

    model = MYClassifier(dropout_prob=0.4, eta=0.001, hidden_dim=300)
    experiment_result = nli.wordentail_experiment(train_data=wordentail_data['train'],
                                  assess_data=wordentail_data['dev'],
                                  model=model,
                                  vector_func=glove_vec,
                                  vector_combo_func=vec_concatenate)
    

# STOP COMMENT: Please do not remove this comment.

Stopping after epoch 111. Training loss did not improve more than tol=1e-05. Final error is 0.07340331608429551.

              precision    recall  f1-score   support

           0      0.884     0.886     0.885      1732
           1      0.403     0.398     0.401       334

    accuracy                          0.807      2066
   macro avg      0.644     0.642     0.643      2066
weighted avg      0.806     0.807     0.807      2066



## Bake-off [1 point]

The goal of the bake-off is to achieve the highest __macro-average F1__ score on a test set that we will make available at the start of the bake-off. The announcement will go out on the discussion forum. To enter, you'll be asked to run `nli.bake_off_evaluation` on the output of your chosen `nli.wordentail_experiment` run. 

The cells below this one constitute your bake-off entry.

The rules described in the [Your original system](#Your-original-system-[3-points]) homework question are also in effect for the bake-off.

Systems that enter will receive the additional homework point, and systems that achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [ ]:
test_data_filename = os.path.join(
    NLIDATA_HOME,
    "bakeoff-wordentail-data",
    "nli_wordentail_bakeoff_data-test.json")

nli.bake_off_evaluation(
    bakeoff_model,
    test_data_filename)


In [42]:
# Enter your bake-off assessment code into this cell.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE
    test_data_filename = os.path.join(
    NLIDATA_HOME,
    "bakeoff-wordentail-data",
    "nli_wordentail_bakeoff_data-test.json")

    nli.bake_off_evaluation(
        experiment_result,
        test_data_filename)

    


              precision    recall  f1-score   support

           0      0.884     0.872     0.878      2036
           1      0.387     0.414     0.400       399

    accuracy                          0.797      2435
   macro avg      0.635     0.643     0.639      2435
weighted avg      0.802     0.797     0.799      2435



In [ ]:
# On an otherwise blank line in this cell, please enter
# your macro-avg f1 value as reported by the code above.
# Please enter only a number between 0 and 1 inclusive.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
    0.64
    